In [ ]:
pip install keras_preprocessing

In [ ]:
import tensorflow as tf
import keras.preprocessing
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import shutil
TRAINING_DIR = "D:\sem_6_folders\TARP\Project\\new\Train"

#for data augmentation and preprocessing
training_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.15, horizontal_flip=True, fill_mode='nearest')

VALIDATION_DIR = "D:\sem_6_folders\TARP\Project\\new\Test"
validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
	TRAINING_DIR,
	target_size=(224,224),
	shuffle = True,
	class_mode='categorical',
  batch_size = 128
)

validation_generator = validation_datagen.flow_from_directory(
	VALIDATION_DIR,
	target_size=(224,224),
	class_mode='categorical',
	shuffle = True,
  batch_size= 14
)

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Dropout

input_tensor = Input(shape=(224, 224, 3))

base_model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(2048, activation='relu')(x)
#dropout to prevent overfitting
x = Dropout(0.25)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(3, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

#to prevent the change of weights during training
for layer in base_model.layers:
    layer.trainable = False

#configure the learning process of the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
for layer in model.layers[:249]:
       layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['acc'])


class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_loss')<=0.1099 and logs.get('loss')<=0.1099):
      print('\n\n Reached The Destination!')
      self.model.stop_training = True
callbacks = myCallback()

history = model.fit(
    train_generator,
    steps_per_epoch = 14,
    epochs = 20,
    validation_data = validation_generator,
    validation_steps = 14,
    callbacks=[callbacks]
)
print(len(base_model.layers))

In [ ]:
%matplotlib inline
from keras.models import load_model
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'g', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.legend(loc=0)
plt.figure()
plt.show()

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'orange', label='Validation loss')
plt.title('Training and validation loss')

plt.legend(loc=0)
plt.figure()
plt.show()

In [3]:
import numpy as np
from PIL import Image
import keras.utils as image
from keras.models import load_model


# file_path = "C:/Users/yveni/Downloads/images5.jpg"
file_path = "D:/sem_6_folders/TARP/Project/new/Test/Fire/image_26.jpg"


model = load_model('model.h5')

img = image.load_img(file_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0) / 255.0  # normalize pixel values between 0 and 1

classes = model.predict(x)
if np.argmax(classes[0]) == 0:
    print("Fire Detected")
    print(np.argmax(classes[0]) == 0, max(classes[0]))
elif np.argmax(classes[0]) == 2:
    print("Smoke Detected")
    print(np.argmax(classes[0]) == 2, max(classes[0]))
else:
    print("No Fire or Smoke Detected")
    # print(np.argmax(classes[0]) == 1, max(classes[0]))

1/1 [==============================] - 2s 2s/step
Fire Detected
True 0.9999995


In [ ]:
model.save('D:/sem_6_folders/TARP/Project/new/model.h5')

In [1]:
pip install pyserial

Note: you may need to restart the kernel to use updated packages.


In [12]:
from pyfirmata import Arduino
import time

# connect to the Arduino board
board = pyfirmata.Arduino('COM3') # Replace COM3 with the port your board is connected to

# set up the digital pin 13 as output
led_pin = board.get_pin('d:8:o')

# turn the LED on
led_pin.write(1)

# wait for 1 second
time.sleep(1)

# turn the LED off
led_pin.write(0)

# close the connection to the board
board.exit()




In [1]:
import numpy as np
from PIL import Image
import tensorflow as tf
import keras.utils as image
import cv2
# from pyfirmata import Arduino
# import time

# # Define the port and board



model = tf.keras.models.load_model('model.h5')

video = cv2.VideoCapture(0)

while True:
    # Capture the current frame from the video stream
    ret, frame = video.read()

    # Convert the captured frame into RGB
    im = Image.fromarray(frame, 'RGB')

    # Resize the image to 224x224, which is the input size expected by the model
    im = im.resize((224,224))

    # Convert the image to a numpy array and scale the pixel values to [0, 1]
    img_array = image.img_to_array(im)
    img_array = np.expand_dims(img_array, axis=0) / 255

    # Use the model to predict the probability of each class (fire or no fire)
    classes = model.predict(img_array)

    # # Initialize the board
    # board = Arduino('COM5')

    # # Set pin 13 as output
    # pin13 = board.get_pin('d:8:o')

    # Add a text overlay to the video frame indicating whether fire is detected or not
    if np.argmax(classes[0]) == 0:
        cv2.putText(frame, "Fire Detected", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
       # Turn the LED on
        # board.digital.write(1)
        # time.sleep(1)

        # # Turn the LED off
        # board.digital.write(0)

        # # Release the board
        # board.exit()
        # cv2.putText(frame, np.argmax(classes[0]) == 0, max(classes[0]), (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    elif np.argmax(classes[0]) == 2:
        cv2.putText(frame, "Smoke Detected", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    else:
        cv2.putText(frame, "No Fire or Smoke Detected", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the video frame in a window
    cv2.imshow("Fire and Smoke Detection", frame)

    # Exit the loop if the user presses 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close the window
video.release()
cv2.destroyAllWindows()





1/1 [==============================] - 0s 101ms/step
